In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [3]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,study room,servant room,store room,furnishing_type,floor_category,price
0,1.0,13.0,5.0,5.0,4.0,1.0,3600.00,1.0,1.0,1.0,2.0,2.0,8.75
1,1.0,57.0,4.0,4.0,3.0,0.0,2700.00,0.0,1.0,1.0,2.0,2.0,6.40
2,0.0,58.0,4.0,4.0,4.0,0.0,2383.00,0.0,1.0,0.0,0.0,2.0,2.90
3,0.0,2.0,3.0,3.0,4.0,0.0,2089.00,0.0,1.0,0.0,0.0,2.0,0.90
4,0.0,18.0,3.0,3.0,2.0,3.0,2225.56,0.0,0.0,0.0,2.0,0.0,3.40


In [6]:
# one hot encode -> sector, balcony, agePossession, furnishing type, floor category

In [7]:
X = df.drop(columns=['price'])
y = df['price']

In [8]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [9]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'floor_category']

In [10]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [16]:
# # Creating a column transformer for preprocessing
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
#         ('cat', OneHotEncoder(drop='first'), columns_to_encode)
#     ], 
#     remainder='passthrough'
# )

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), columns_to_encode)
    ])

In [17]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [18]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [19]:
scores.mean()

0.8746767995733575

In [20]:
scores.std()

0.024362001152441345

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [22]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [23]:
y_pred = pipeline.predict(X_test)

In [24]:
y_pred = np.expm1(y_pred)

In [25]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5712594624063736

In [30]:
# Model Avg error is 57-58 lacks

# More to do
- Use different Algorithms
- hyperparameter tuning
    